In [80]:
import numpy as np
import logging 
import matplotlib.pyplot as plt 
import copy

In [ ]:
# logging.config()

In [ ]:
def ReLU(x,leaky=False,negative_slope=None): # leaky is a flag for leaky relu, negative slope for leaky relu. 
    if leaky and negative_slope!=None:
        try:
            if negative_slope>0:
                raise ValueError("Negative slope accepts integers only.")            
            return np.where(x<0,x*negative_slope,x)
        
        except ValueError as ve:
            print(f"an unexpected error{ve}")
            
        except Exception as e:            
            print(f"An unexpected error {e}")
            raise 
    return np.maximum(0,x)


In [ ]:
def gradient_ReLU(x):
    return np.where(x>0,1,0)

In [54]:
def normal_initialization(fraction,weight_size):
    fract=np.sqrt(2/fraction)
    return np.random.normal(0,fract,size=weight_size)

In [55]:
def uniform_initialization(fraction,weight_size):
    fract= np.sqrt(6/fraction)
    return np.random.uniform(-fract,fract,size=weight_size)


In [56]:
method_map={
        "uniform":uniform_initialization,
        "normal":normal_initialization
    }

In [98]:
def param_initialize(layer_in,layer_out,mode="he",method="uniform",bias=True):
    """ Returns weights and biases initialized as response to input information

    Args:
        layer_in (_type_): _description_
        layer_out (_type_): _description_
        mode (str, optional): _description_. Defaults to "he".
        method (str, optional): _description_. Defaults to "uniform".

    Raises:
        ValueError: _description_
    """
    
    
    
    weights=np.zeros((layer_in,layer_out))
    if bias:
        biases=np.zeros((1,layer_out))
    
    
    try:        
        calcluation=0
        
        if mode.lower()=='random': return np.random.random(size=(layer_in,layer_out))
        
        elif mode.lower()=="he":
            calcluation=1/(np.sqrt(layer_in+layer_out))
            
        elif mode.lower()=="xavier":
            calcluation=1/(np.sqrt(layer_in))
            
        else:
            raise ValueError("Only accepts 'random','he' and 'xavier' string as arguments.")

        weights=method_map[method](calcluation,(layer_in,layer_out))
        
        
       
    except Exception as e:
        print(f"Error occured: {e}")    

    
    return {"weight":weights,"bias":biases}
    

In [107]:
layer=(2,3)
val=param_initialize(2,3,method="normal")
val['weight']


array([[ 6.87669625,  1.39198511,  2.39157925],
       [-1.09132129,  2.83037356, -2.77389239]])

In [149]:
custom_hidden_layers=[8,4,2,1]
custom_X=np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15]]) # 5 x 3 

In [169]:
def build_network(hidden_layers,X):
    neural_network={}
    new_hidden=hidden_layers.copy()
    new_hidden.insert(0,X.shape[1])
    for layer in range(len(hidden_layers)):
        neural_network[f"Layer{layer}"]=param_initialize(new_hidden[layer],new_hidden[layer+1])
    return neural_network
    

In [175]:
network=build_network(hidden_layers=custom_hidden_layers,X=custom_X)
network

{'Layer0': {'weight': array([[ 2.36789228,  3.563086  , -3.1362556 , -3.26887007, -4.15671336,
           2.06487899, -3.885151  , -3.60701948],
         [ 1.59679756, -3.17172828, -0.9273428 ,  3.73348393, -0.08931366,
           1.52917609, -1.16050119,  0.48777265],
         [-2.15462478,  3.3232668 , -2.91558806, -4.03043921, -3.23013273,
           1.28411053, -3.44819656,  1.8992314 ]]),
  'bias': array([[0., 0., 0., 0., 0., 0., 0., 0.]])},
 'Layer1': {'weight': array([[ 0.47238024,  1.8910155 ,  1.08811856, -1.71385816],
         [-1.029656  ,  0.2821889 ,  1.05560328,  3.67489456],
         [ 4.31650387, -4.53515878, -2.65513398, -4.28496947],
         [ 1.7994913 ,  0.10917044, -3.15688514,  0.16796728],
         [ 1.2407165 ,  1.54807985, -3.2153055 , -0.54417098],
         [ 2.39103817, -1.10708266,  0.03788637, -1.93473997],
         [-3.74980262,  2.39401258, -3.13078658, -0.19942737],
         [-1.96082426,  3.64753117,  0.19755342,  0.77439823]]),
  'bias': array([[0., 0

In [176]:
network['Layer0']["weight"]

array([[ 2.36789228,  3.563086  , -3.1362556 , -3.26887007, -4.15671336,
         2.06487899, -3.885151  , -3.60701948],
       [ 1.59679756, -3.17172828, -0.9273428 ,  3.73348393, -0.08931366,
         1.52917609, -1.16050119,  0.48777265],
       [-2.15462478,  3.3232668 , -2.91558806, -4.03043921, -3.23013273,
         1.28411053, -3.44819656,  1.8992314 ]])

In [177]:
def forward_pass(input_network,X):
    network=input_network.copy()
    result=X @ network['Layer0']["weight"] + network['Layer0']["bias"]
    network.pop('Layer0')
    for weight,bias in network.items():
        result=result @ weight + bias
        
    return result 

In [179]:
forward_pass(network,X=custom_X)

UFuncTypeError: ufunc 'matmul' did not contain a loop with signature matching types (dtype('float64'), dtype('<U6')) -> None